In [1]:
def calc(op, num1, num2=None):
    
    if not isinstance(num1, (int, float)):
        raise Exception(f"Invalid number \"{num1}\"")
    if num2 is not None and not isinstance(num2, (int, float)):
        raise Exception(f"Invalid number \"{num2}\"")

    if op not in ["+", "-", "*", "/", "%", "add", "sub", "mul", "div", "mod", "pow"]:
        raise Exception(f"Invalid operator \"{op}\"")

    if num2 is None:
        if op == "+" or op == "add":
            return num1
        elif op == "-" or op == "sub":
            return -num1
        else:
            raise Exception("This operator requires two numbers")

    if op == "+" or op == "add":
        return num1 + num2
    elif op == "-" or op == "sub":
        return num1 - num2
    elif op == "*" or op == "mul":
        return num1 * num2
    elif op == "/" or op == "div":
        if num2 == 0:
            raise Exception("Division by zero")
        return num1 / num2
    elif op == "%" or op == "mod":
        if num2 == 0:
            raise Exception("Division by zero")
        return num1 % num2
    elif op == "^" or op == "pow":
        return num1 ** num2

In [2]:
def eval(expression):

    if not isinstance(expression, list) or not expression:
        return f"Failed to evaluate \"{expression}\""

    for i, item in enumerate(expression):
        if isinstance(item, list):
            expression[i] = eval(item)
    
    if len(expression) == 1:
        return expression[0]
    
    try:
        operator = expression[0]

        if len(expression) == 2:
            return calc(operator, expression[1])

        elif len(expression) == 3:
            return calc(operator, expression[1], expression[2])
        else:
            raise Exception(f"Failed to evaluate \"{expression}\"")
    except Exception as e:
        raise Exception(f"Failed to evaluate \"{expression}\"")

In [3]:
def struct(expression):
    if not expression:
        return []
    if isinstance(expression, list) == False:
        raise Exception(f"Failed to structure \"{expression}\"")
    if len(expression) == 1:
        raise Exception(f"Failed to structure \"{expression}\"")
    if len(expression) == 2:
        return expression
        
    # Count the number of non-numeric operators (excluding lists)
    num_op = sum(1 for item in expression 
                 if not isinstance(item, (int, float, list)))
    nums = len(expression) - num_op
    
    if abs(num_op - nums) > 1 or num_op - nums == 0:
        raise Exception(f"Failed to structure \"{expression}\"")
    
    # Definimos operadores por niveles
    level0_ops = {"^", "pow"}  # Mayor precedencia, asociativo por la derecha
    level1_ops = {'*', '/', '%', 'mul', 'div', 'mod'}  # Precedencia media, asociativo por la izquierda
    level2_ops = {'+', '-', 'add', 'sub'}  # Menor precedencia, asociativo por la izquierda

    def parse_expression(expression, operators, right_associative=False):
        if right_associative:
            # Para operadores asociativos por la derecha, buscamos el primer operador desde la derecha
            i = len(expression) - 2
            while i >= 1:
                # Check if current item is an operator (not a number or list)
                if isinstance(expression[i], str) and expression[i] in operators:
                    # Agrupamos directamente los dos números alrededor del operador
                    left = expression[i-1]
                    op = expression[i]
                    right = expression[i+1]
                    # Reemplazamos la subexpresión por el resultado agrupado
                    new_expression = expression[:i-1] + [[op, left, right]] + expression[i+2:]
                    return parse_expression(new_expression, operators, right_associative)
                i -= 2
        else:
            # Para operadores asociativos por la izquierda, buscamos el primer operador desde la izquierda
            i = 1
            while i < len(expression):
                # Check if current item is an operator (not a number or list)
                if isinstance(expression[i], str) and expression[i] in operators:
                    # Agrupamos directamente los dos números alrededor del operador
                    left = expression[i-1]
                    op = expression[i]
                    right = expression[i+1]
                    # Reemplazamos la subexpresión por el resultado agrupado
                    new_expression = expression[:i-1] + [[op, left, right]] + expression[i+2:]
                    return parse_expression(new_expression, operators, right_associative)
                i += 2

        # Si no hay operadores, devolvemos la expresión tal cual
        return expression[0] if len(expression) == 1 else expression

    # Primero procesamos operadores de nivel 0 (mayor precedencia, asociativo por la derecha)
    expression = parse_expression(expression, level0_ops, right_associative=True)

    # Luego procesamos operadores de nivel 1 (precedencia media, asociativo por la izquierda)
    if isinstance(expression, list):
        expression = parse_expression(expression, level1_ops)

    # Procesamos los operadores de nivel 2 (menor precedencia, asociativo por la izquierda)
    if isinstance(expression, list):
        expression = parse_expression(expression, level2_ops)

    # Finalmente, procesamos los operadores no válidos de izquierda a derecha
    while isinstance(expression, list):
        # Encuentra el primer operador no válido de izquierda a derecha
        invalid_ops = [i for i in range(len(expression)) 
                       if isinstance(expression[i], str) and 
                       expression[i] not in level0_ops.union(level1_ops).union(level2_ops)]
        
        if not invalid_ops:
            break
        
        # Tomamos el operador más a la izquierda
        i = min(invalid_ops)
        
        # Agrupamos alrededor de este operador
        left = expression[i-1]
        op = expression[i]
        right = expression[i+1]
        expression = expression[:i-1] + [[op, left, right]] + expression[i+2:]

    # Aseguramos que el resultado final no esté envuelto en una lista adicional
    return expression[0] if isinstance(expression, list) and len(expression) == 1 else expression

In [4]:
def get_next(s, start):
    #Error handling 
    if len(s) == 0 or start >= len(s):
        raise Exception("End of string")
    # Defined operators for reference
    all_ops = ["^", "pow", '*', '/', '%', 'mul', 'div', 'mod',    '+', '-', 'add', 'sub']
    # Check if start index is on an operator
    if not s[start].isdigit() and not s[start].isalpha():
        return s[start]
    elif s[start].isalpha():
        index = start
        op = ""
        while index !=  len(s) and  s[index].isalpha():
            op += s[index]
            index += 1
        return op
    # Extract number
    index = start
    number = ''
    while index < len(s) and (s[index].isdigit() or s[index] == '.'):
        number += s[index]
        index += 1

    return float(number) if '.' in number else int(number)

In [5]:
def parse(s):
    s = s.replace(" ", "")
    ops_nums = []
    while len(s) > 0:
        next_num_op = get_next(s, 0)
        
        if next_num_op == "(":
            p_index = s.find(")")
            next_num_op = parse(s[1:p_index])
            s = s[p_index:].replace(")", "", 1)
            
        s = s.replace(str(next_num_op), "", 1)
        ops_nums.append(next_num_op)

    return struct(ops_nums)

In [ ]:
def pre_parse(s):
    s = [par for par in s if par == "(" or par == ")"]
    open_par = 0
    for par in s:
        if open_par < 0:
            raise Exception('Not matching parenthesis')
        if par == "(":
            open_par += 1
        if par == ")":
            open_par -= 1
    if open_par != 0:
        raise Exception('Not matching parenthesis')

In [ ]:
def coordinate(s):
    try:
        pre_parse(s)
        result = eval(parse(s))
        return result
    except ValueError as e:
        return f"Error: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"